In [1]:
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, InceptionV3, MobileNet, EfficientNetB0

In [2]:
# Get the current working directory
# cwd = os.getcwd()

X, y = [], []
path = "wheat_leaf"
class_dict = {'Healthy':0, 
              'septoria':1, 
              'stripe_rust':2}

for each_class in class_dict.keys():
    # Get all the files in the path directory
    files = os.listdir(path + '/' + each_class)
    # print(files)
    for each_file in files:
        if each_file[0] == '.':
            continue
            
#         image = tf.keras.utils.load_img(path + '/' + each_class + '/' + each_file, 
#                                         target_size=(224, 224))
#         image_arr = tf.keras.utils.img_to_array(image)

        img = cv.imread(path + '/' + each_class + '/' + each_file)
        image_arr = cv.resize(img, (224,224))
        X.append(image_arr)
        y.append(class_dict[each_class])
    print('successfully loaded for class:',  each_class)

successfully loaded for class: Healthy
successfully loaded for class: septoria
successfully loaded for class: stripe_rust


In [3]:
X, y = np.array(X), np.array(y)
unique_values, counts = np.unique(y, return_counts=True)
X.shape, y.shape, unique_values, counts, (counts / len(y) * 100).round(2)

((407, 224, 224, 3),
 (407,),
 array([0, 1, 2]),
 array([102,  97, 208], dtype=int64),
 array([25.06, 23.83, 51.11]))

In [4]:
# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the image width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the image height
    zoom_range=0.2,         # Random zoom by up to 20%
    horizontal_flip=True,   # Random horizontal flipping
    fill_mode='nearest'
)

In [5]:
new_X = np.empty((0, 224, 224, 3))
new_y = np.empty((0, ))
print("Applying data augmentation with each_class Target 600 \n")
for each_class in class_dict:
    print("Applying data augmentation for : ", each_class)
    list_index = np.where(y == class_dict[each_class])
    list_imgs = X[list_index]
    print(f"No of Images available for {each_class} is {list_imgs.shape[0]}")
    target, available = 600, list_imgs.shape[0]
    imgs, req_batch = list_imgs, target - available
    print(f"No of Images required for {each_class} is {req_batch} \n")
    while req_batch > 0 :
        new_imgs = train_datagen.flow(list_imgs, batch_size=req_batch)
        tmp_imgs = new_imgs.next()
        req_batch = req_batch - available
        imgs = np.concatenate((np.array(imgs), np.array(tmp_imgs, dtype='uint8')))

    ys = np.zeros(600) + class_dict[each_class]
    new_X = np.concatenate((new_X, imgs))
    new_y = np.concatenate((new_y, ys))

Applying data augmentation with each_class Target 600 

Applying data augmentation for :  Healthy
No of Images available for Healthy is 102
No of Images required for Healthy is 498 

Applying data augmentation for :  septoria
No of Images available for septoria is 97
No of Images required for septoria is 503 

Applying data augmentation for :  stripe_rust
No of Images available for stripe_rust is 208
No of Images required for stripe_rust is 392 



In [6]:
new_X, new_y = np.array(new_X), np.array(new_y)
unique_values, counts = np.unique(new_y, return_counts=True)
new_X.shape, new_y.shape, unique_values, counts, (counts / len(new_y) * 100).round(2)

((1800, 224, 224, 3),
 (1800,),
 array([0., 1., 2.]),
 array([600, 600, 600], dtype=int64),
 array([33.33, 33.33, 33.33]))

In [7]:
X_train, X_dummy, y_train, y_dummy = train_test_split(new_X, new_y, train_size=0.9, shuffle=True, random_state=123, stratify=new_y)
print(f'training',X_train.shape, X_dummy.shape, y_train.shape, y_dummy.shape)

X_val, X_test, y_val, y_test = train_test_split(X_dummy, y_dummy, train_size=0.5, shuffle=True, random_state=123, stratify=y_dummy)
print(f'val - test',X_val.shape, X_test.shape, y_val.shape, y_test.shape)

training (1620, 224, 224, 3) (180, 224, 224, 3) (1620,) (180,)
val - test (90, 224, 224, 3) (90, 224, 224, 3) (90,) (90,)


In [8]:
X_train_, y_train_, X_val_, X_test_, y_val_, y_test_ = X_train, y_train, X_val, X_test, y_val, y_test

In [9]:
X_train = X_train / 255
X_test = X_test / 255
X_val = X_val / 255

In [10]:
# y_train.value_counts()
unique_values, counts = np.unique(y_train, return_counts=True)
unique_values, counts, (counts / len(y_train) * 100).round(2)

(array([0., 1., 2.]),
 array([540, 540, 540], dtype=int64),
 array([33.33, 33.33, 33.33]))

In [11]:
# y_test.value_counts()
unique_values, counts = np.unique(y_test, return_counts=True)
unique_values, counts, (counts / len(y_test) * 100).round(2)

(array([0., 1., 2.]),
 array([30, 30, 30], dtype=int64),
 array([33.33, 33.33, 33.33]))

In [12]:
# y_val.value_counts()
unique_values, counts = np.unique(y_val, return_counts=True)
unique_values, counts, (counts / len(y_test) * 100).round(2)

(array([0., 1., 2.]),
 array([30, 30, 30], dtype=int64),
 array([33.33, 33.33, 33.33]))

In [13]:
# Convert labels to one-hot encoding
num_classes = 3
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_train.shape, y_test.shape, y_val.shape

((1620, 3), (90, 3), (90, 3))

In [14]:
def create_custom_model(base_model, input_shape, num_classes):
    # Freeze the pretrained convolutional base
    base_model.trainable = False
    
    # Add custom classification layers on top of the base model
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the full model
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model


In [15]:
# Define input shape and number of classes
input_shape = (224, 224, 3)  
num_classes = 3  

# Load the pretrained models with include_top=False
vgg19_base = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
inceptionv3_base = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
mobilenet_base = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
efficientnet_base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

In [16]:
# Create custom models(0108) using each base model
vgg19_0108_model = create_custom_model(vgg19_base, input_shape, num_classes)
inceptionv3_0108_model = create_custom_model(inceptionv3_base, input_shape, num_classes)
mobilenet_0108_model = create_custom_model(mobilenet_base, input_shape, num_classes)
efficientnet_0108_model = create_custom_model(efficientnet_base, input_shape, num_classes)

# Compile the model
vgg19_0108_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inceptionv3_0108_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
mobilenet_0108_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
efficientnet_0108_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Print the summaries of each custom model
vgg19_0108_model.summary()
inceptionv3_0108_model.summary()
mobilenet_0108_model.summary()
efficientnet_0108_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
# Train the model
vgg19_model_history = vgg19_0108_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
26/26 [==============================] - 380s 14s/step - loss: 0.9422 - accuracy: 0.5574 - val_loss: 0.6788 - val_accuracy: 0.7444
Epoch 2/10
26/26 [==============================] - 469s 18s/step - loss: 0.6146 - accuracy: 0.7710 - val_loss: 0.5250 - val_accuracy: 0.8000
Epoch 3/10
 2/26 [=>............................] - ETA: 6:30 - loss: 0.5089 - accuracy: 0.8203

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
vgg19_test_loss, vgg19_test_acc = vgg19_0108_model.evaluate(X_test, y_test)

print('Test loss:', vgg19_test_loss)
print('Test accuracy:', vgg19_test_acc)

3/3 [==============================] - 13s 5s/step - loss: 0.1544 - accuracy: 0.9444
Test loss: 0.15436427295207977
Test accuracy: 0.9444444179534912


In [ ]:
inceptionv3_model_history = inceptionv3_0108_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
26/26 [==============================] - 85s 3s/step - loss: 0.7478 - accuracy: 0.7130 - val_loss: 0.2997 - val_accuracy: 0.8778
Epoch 2/10
26/26 [==============================] - 82s 3s/step - loss: 0.2570 - accuracy: 0.8988 - val_loss: 0.2814 - val_accuracy: 0.8889
Epoch 3/10
26/26 [==============================] - 79s 3s/step - loss: 0.1896 - accuracy: 0.9235 - val_loss: 0.2448 - val_accuracy: 0.9000
Epoch 4/10
26/26 [==============================] - 75s 3s/step - loss: 0.1548 - accuracy: 0.9389 - val_loss: 0.1749 - val_accuracy: 0.9111
Epoch 5/10
26/26 [==============================] - 198s 8s/step - loss: 0.1363 - accuracy: 0.9469 - val_loss: 0.1709 - val_accuracy: 0.9111
Epoch 6/10
26/26 [==============================] - 57s 2s/step - loss: 0.1040 - accuracy: 0.9623 - val_loss: 0.1613 - val_accuracy: 0.9000
Epoch 7/10
26/26 [==============================] - 56s 2s/step - loss: 0.0751 - accuracy: 0.9710 - val_loss: 0.1800 - val_accuracy: 0.9222
Epoch 8/10
26/26 [=

In [ ]:
# Evaluate the model
inceptionv3_test_loss, inceptionv3_test_acc = inceptionv3_0108_model.evaluate(X_test, y_test)

print('Test loss:', inceptionv3_test_loss)
print('Test accuracy:', inceptionv3_test_acc)

3/3 [==============================] - 2s 643ms/step - loss: 0.0428 - accuracy: 0.9778
Test loss: 0.042842019349336624
Test accuracy: 0.9777777791023254


In [ ]:
mobilenet_model_history = mobilenet_0108_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
26/26 [==============================] - 28s 1s/step - loss: 0.4986 - accuracy: 0.7901 - val_loss: 0.1751 - val_accuracy: 0.9556
Epoch 2/10
26/26 [==============================] - 26s 995ms/step - loss: 0.1478 - accuracy: 0.9432 - val_loss: 0.1102 - val_accuracy: 0.9778
Epoch 3/10
26/26 [==============================] - 26s 1s/step - loss: 0.0727 - accuracy: 0.9802 - val_loss: 0.1068 - val_accuracy: 0.9778
Epoch 4/10
26/26 [==============================] - 27s 1s/step - loss: 0.0592 - accuracy: 0.9796 - val_loss: 0.0838 - val_accuracy: 0.9778
Epoch 5/10
26/26 [==============================] - 26s 1s/step - loss: 0.0318 - accuracy: 0.9944 - val_loss: 0.0957 - val_accuracy: 0.9667
Epoch 6/10
26/26 [==============================] - 26s 1s/step - loss: 0.0335 - accuracy: 0.9932 - val_loss: 0.0534 - val_accuracy: 0.9778
Epoch 7/10
26/26 [==============================] - 26s 1s/step - loss: 0.0183 - accuracy: 0.9969 - val_loss: 0.0407 - val_accuracy: 0.9778
Epoch 8/10
26/26 

In [ ]:
# Evaluate the model
mobilenet_test_loss, mobilenet_test_acc = mobilenet_0108_model.evaluate(X_test, y_test)

print('Test loss:', mobilenet_test_loss)
print('Test accuracy:', mobilenet_test_acc)

3/3 [==============================] - 2s 492ms/step - loss: 0.0182 - accuracy: 0.9889
Test loss: 0.0181741826236248
Test accuracy: 0.9888888597488403


In [ ]:
efficientnet_model_history = efficientnet_0108_model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))

In [ ]:
# Evaluate the model
efficientnet_test_loss, efficientnet_test_acc = efficientnet_0108_model.evaluate(X_test, y_test)

print('Test loss:', efficientnet_test_loss)
print('Test accuracy:', efficientnet_test_acc)

In [ ]:
# efficientnet_model_history, vgg19_model_history, inceptionv3_model_history, mobilenet_model_history
import pandas as pd
df = pd.concat(
    [pd.DataFrame(vgg19_model_history.history).reset_index(names='Epoch'),
    pd.DataFrame(inceptionv3_model_history.history).reset_index(names='Epoch'),
    pd.DataFrame(mobilenet_model_history.history).reset_index(names='Epoch')],
    keys=['vgg19', 'inceptionv3', 'mobilenet'],
    axis = 1
)
df

vgg19                                            inceptionv3            \
  Epoch      loss  accuracy  val_loss val_accuracy       Epoch      loss   
0     0  0.942443  0.541975  0.686171     0.755556           0  0.747786   
1     1  0.608719  0.775926  0.519954     0.866667           1  0.256954   
2     2  0.495858  0.820370  0.474501     0.855556           2  0.189631   
3     3  0.402737  0.856790  0.365317     0.888889           3  0.154837   
4     4  0.344105  0.881481  0.316755     0.911111           4  0.136270   
5     5  0.292187  0.909877  0.285847     0.922222           5  0.103983   
6     6  0.264614  0.911728  0.282802     0.922222           6  0.075057   
7     7  0.244426  0.913580  0.302734     0.877778           7  0.072514   
8     8  0.224516  0.924074  0.233268     0.933333           8  0.078104   
9     9  0.199563  0.933333  0.212159     0.933333           9  0.067900   

                                   mobilenet                                \
   accuracy  val_loss val_accuracy     Epoch      loss  accuracy  val_loss   
0  0.712963  0.299689     0.877778         0  0.498626  0.790123  0.175137   
1  0.898765  0.281368     0.888889         1  0.147837  0.943210  0.110221   
2  0.923457  0.244791     0.900000         2  0.072664  0.980247  0.106840   
3  0.938889  0.174919     0.911111         3  0.059151  0.979630  0.083795   
4  0.946914  0.170905     0.911111         4  0.031847  0.994444  0.095710   
5  0.962346  0.161292     0.900000         5  0.033516  0.993210  0.053443   
6  0.970988  0.180005     0.922222         6  0.018325  0.996914  0.040698   
7  0.972222  0.150564     0.911111         7  0.013864  0.999383  0.041481   
8  0.972840  0.161149     0.933333         8  0.014668  0.996914  0.043450   
9  0.974074  0.200483     0.922222         9  0.012863  0.997531  0.033937   

                
  val_accuracy  
0     0.955556  
1     0.977778  
2     0.977778  
3     0.977778  
4     0.966667  
5     0.977778  
6     0.977778  
7     0.977778  
8     0.977778  
9     0.988889